### Sets and Indices
$j \in J = \{1,2,...,n\}$: Index and set of jobs.

$k$: UUV sub

$d \in D = \{n+1, n+2, ...,n+m \}$: Index and set of depots (service centers), where $m$ is the number of depots. (NOT SURE IF THIS EVER GETS USED)

$l, i, j \in L = J \cup D = \{1,2,..., n+m \}$: index and set of locations to visit.

$\lambda_i \in i = {1, 2, ..., n}$: sensory learning rate at each hotspot

### Parameters
$p_{j} \in \mathbb{R^{+}}$; Duration of job $j \in J$.

$\tau_{i,j} \in \mathbb{R^{+}}$: Travel time, in minutes, from location $i \in L$ to location $j \in L$.

$W \in \mathbb{N}$: The maximum number of hours in minutes that the UUV can survey for.

$\pi_{j} \in \{1,2,3,4\}$: Amount of trash expected to gather at job $j \in J$, a larger number means higher priority.

$a_{j} \in \mathbb{R^{+}} $: Earliest time to start the service at location of job $j \in J$.

$b_{j} \in \mathbb{R^{+}} $: Latest time to start the service at location of job $j \in J$.

$M \in \mathbb{N}$: This is a very large number. This number is determined as follows: the planning horizon is of 10 working hours (i.e. 600 min). We want the value of $M$ to be an order of magnitude larger than the planning horizon. Therefore, we set M = 6100.

### Decision Variables

$x_{j} \in \{0,1\}$: This variable is equal 1 if job $j \in J$ is assigned to uuv $k$ at timeslot $t$, and 0 otherwise.

$y_{i,j,t} \in \{0,1\}$: This variable is equal 1 if uuv $k$  travels from location $i \in L$ to location 
$j \in L$ at timeslot $t$; and 0 otherwise.

$t_{j} \geq 0$: This variable determines the time to arrive or start the service at location $j \in J$.

$xa_{j}, xb_{j} \geq 0$: Correction to earliest and latest time to start the service for job $j \in J$.

$g_{j}$: This variable is equal 1 if job $j \in J$ cannot be filled, and 0 otherwise. 

### Objective Function

- **Maximize profit** The Objective function is to maximize the amount of profit (ie how much trash we can collect). 

<!-- \begin{equation}
\sum_{j \in J} \pi_{j} \cdot z_{j} + \sum_{j \in J} 0.01 \cdot \pi_{j} \cdot M(xa_{j} + xb_{j}) + 
\sum_{j \in J} \pi_{j} \cdot M \cdot g_{j}
\tag{0}
\end{equation} -->

\begin{equation}
\max \sum^{N-1}_{i=2} \sum^N_{j=2} \sum^{t_{ij}}_{t=1} P_i x_{ijt} f_i(p_j), where f_i(p_j) = 1 - e^{-\lambda_i p_j} 0 \leq t_i \leq +\inf
\tag{0}
\end{equation}


### Constraints

- **Ensure hotspot is only visited once:** For each job, we ensure the hotspot is visited, or a gap is declared.  **there are better ways of doing this

\begin{equation}
\sum_{j \in J} x_{j} + g_{j} = 1 \quad \forall j \in J
\tag{1}
\end{equation}

**Note**: The penalty of the gap variable $g_{j}$ is ($0.1 \cdot \pi_{j} \cdot M$) which is a large number to discourage 
not being able to satisfy demand.

<!-- - **Only one technician:** For each job, we only allow one technician to be assigned.

 \begin{equation}
 \sum_{k \in K} x_{j,k} \leq 1 \quad \forall j \in J
 \tag{2}
 \end{equation}
 -->
- **Technician capacity:** For each technician, we ensure that the available capacity of the technician is not exceeded. 

\begin{equation}
\sum_{j \in J}  p_{j} \cdot x_{j,k} + \sum_{i \in L} \sum_{j \in L} \tau_{i,j} \cdot y_{i,j,k} \leq W_{k} \quad \forall k \in K
\tag{3}
\end{equation}

- **Technician tour:** For each technician and job, we ensure that if the technician is assigned to the job, then the technician must travel to another location (to form a tour). 

\begin{equation}
\sum_{j \in L}  y_{i,j,k} = x_{i,k} \quad \forall i \in J, k \in K
\tag{4}
\end{equation}

- For each technician and job, we ensure that if a technician is assigned to the job, then the technician must travel from another location to the location of the job (to form a tour). 

\begin{equation}
\sum_{i \in L}  y_{i,j,k} = x_{j,k} \quad \forall j \in J, k \in K
\tag{5}
\end{equation}

- **Temporal relationship:** For each location and job, we ensure the temporal relationship between two consecutive jobs served by the same technician. That is, if a technician $k$ travels from job $i$ to job $j$, then the start of the service time at job $j$ must be no less than the completion time of job $i$ plus the travel time from job $i$ to job $j$. 

\begin{equation}
t_{j} \geq t_{i} + p_{i} + \tau_{i,j} - M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) \quad \forall i \in L, j \in J
\tag{8}
\end{equation}

**Note**: Observe that if the technician $k$ travels from the location of job $i$ to the location of job $j$, then
$\sum_{k \in K}  y_{i,j,k} = 1$. Therefore, $M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) = 0$, and the constraint 
$t_{j} \geq t_{i} + p_{i} + \tau_{i,j}$ would be properly enforced. Now consider the case where the technician $k$ does not travel  from the location of job $i$ to the location of job $j$. Hence, $\sum_{k \in K}  y_{i,j,k} = 0$ and 
$M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) = M$. In this case, this constraint becomes 
$t_{j} \geq t_{i} + p_{i} + \tau_{i,j} - M$. But $M$ is a very large number, then $t_{i} + p_{i} + \tau_{i,j} - M < 0$ and 
since $t_{j} \geq 0$, this constraint is redundant.

- **Time window:** For each job $j \in J$ ensure that the time window for the job is satisfied.

\begin{equation}
t_{j} \geq a_{j} - xa_{j} \quad \forall j \in J
\tag{9}
\end{equation}

\begin{equation}
t_{j} \leq b_{j} + xb_{j} \quad \forall j \in J
\tag{10}
\end{equation}

**Note**: To discourage that the time window of a job is violated, we associate the  penalty of 
($0.01 \cdot \pi_{j} \cdot M$) to the correction variables $xa_{j}, xb_{j}$.

### Sets and Indicies

$n_i = {1,2,..., N}$ where $i \in N$ is the set of nodes/hotspots to visit

$P_i$ : non-negative density of trash expected at each hotspot $i$ (higher density = more desirable)

$\lambda_i$ : sensory learning rate, given by $\frac{obs_r^2 \pi}{A_i}$, where $obs_r^2 \pi$ = UUV sensing rate, and $A_i$ = area of hotspot

$t_i$ : total amount of time spent at hotspot $i$

$h = {0, 0.5, 1, 1.5, ... , T_{max}}$ : timeslot that the path from hotspot $i$ to $j$ is taken at. Timeslots are every 30 minutes from 0 to $T_{max}$

$[O_i, C_i]$ : Start and end times to spend at the node

$d_{i,j,t}$ : non-negative travel time/cost between node $i$ to $j$ during timeslot $t$

$[0, T_{max}]$ : start and end times are assigned this time window to limit time to $T_{max}$. Service times are included in the travel times

$x_{ijh} \in \{0,1\}$ : 1 if uuv travels from node $i$ to $j$ during timeslot h

$x_n$ : 1 if hotspot is visited by the uuv

$s_i$ : start time of the service at node $i$

$L$ : Large constant



### Objective function

- **Maximize profit** The Objective function is to maximize the amount of profit (ie how much trash we can collect across all timeslots). 

\begin{equation}
\max \sum^{N-1}_{i=2}\sum^{N}_{j=2} \sum^{T_{max}}_{h=0} P_i x_{ijh} f_i(t_i)
\tag{0}
\end{equation}

where

\begin{equation}
f_i(t_i) = 1 - e^{-\lambda_i t_i} 0 \leq t_i \leq +\infty
\tag{1}
\end{equation}

The goal here is to determine which hotspots to visit, in what order, for how long, and when

### Constraints

**Connectivity**

**A location may only be selected once for all time slots**: If a path is selected to be traveled, make sure the values equal one across all time

\begin{equation}
\sum_{n=1}^N \sum_{h=0}^{T_{max}} x_{n,h} \leq 1
\end{equation}

**A path may only be selected once for all time slots**

\begin{equation}
\sum^{T_{max}}_{h=0} \sum_{i=1}^N \sum_{j=1, j\neq i}^N x_{ijh} \leq 1
\end{equation}


**Ensure there is no return trip to the starting node

Hotspot 1 will not necessarily be the first node selected in this (we will set the start to every one of the nodes), but 



**Restrict start service time to time window (CHECK THIS)**

TODO: how do we change the time windows so that it finds the best time?

TODO: maybe I don't need a time window orienteering problem? Just need an OP problem. Attempt with just OP first and see how it goes.

<!-- \begin{equation}
O_i \leq s_i \quad i=1,...,N
\tag{3}
\end{equation}

\begin{equation}
s_i \leq c_i \quad i=1,...,N
\tag{4}
\end{equation} -->

**Timeline of the route**
\begin{equation}
s_i + t_i + d_{ijh} \leq s_j
\end{equation}

<!-- 
\begin{equation}
s_i + t_{i,j} - s_j \leq L(1 - x_{i,j})
\tag{5}
\end{equation}
 -->
 
**Ensure $T_{max}$ is not exceeded**
\begin{equation}
\sum_{n \in N} t_i \cdot x_n + \sum_{i \in N} \sum_{j \in N} \sum_{h \in T_{max}} d_{ijh} \cdot x_{ijh} \leq T_{max}
\tag{3}
\end{equation}




**Ensure a tour happens (TODO)**

If we travel from i to j at t, then we cannot travel from j to i at a later time (ensure nodes are visited only once)

<!-- **Connectivity** : connectivity of the route and guarantees that each node is visited at most once for all timeslots t

\begin{equation}
\sum^{t_{ij}}_{i=1} \sum^{N-1}_{i=1} x_{i,k,t} = \sum^{t_{ij}}_{i=1} \sum^{N}_{j=2} x_{k,j,t} \leq 1 \quad k=2,..., N-1
\tag{2}
\end{equation}

**Ensure a tour** -->
- **Technician tour:** For each technician and job, we ensure that if the technician is assigned to the job, then the technician must travel to another location (to form a tour). 

\begin{equation}
\sum_{j \in L}  y_{i,j,k} = x_{i,k} \quad \forall i \in J, k \in K
\tag{4}
\end{equation}

- For each technician and job, we ensure that if a technician is assigned to the job, then the technician must travel from another location to the location of the job (to form a tour). 

\begin{equation}
\sum_{i \in L}  y_{i,j,k} = x_{j,k} \quad \forall j \in J, k \in K
\tag{5}
\end{equation}

(flow balance constraints)
Set edge as traveled if selected. This value should match the total number of hotspots visited (ensures no subtours). Ensure that the number of predecessors is equal to the number of successors for each task in the solution

\begin{equation}
\sum_{i \in N} \sum_{j \in N} \sum_{h \in T_{max}} x_{ijh} = \sum_{i \in N} \sum_{j \in N} \sum_{h \in T_{max}} x_{jih} = \sum_{i \in N} x_n
\end{equation}

## Hotspot example data
    
| Hotspot | 1 | 2 | 3 | 4 | 5 | 6 |
| --- | --- | --- | --- | --- | --- | --- |
| $P_i$ | 480 | 480 | 480 | 480 | 480 | 360 |
| $A_i$ | 480 | 480 | 480 | 480 | 480 | 360 |
| $\lambda_i$ | 480 | 480 | 480 | 480 | 480 | 360 |


Setup all the variables

$n_i = {1,2,..., N}$ where $i \in N$ is the set of nodes/hotspots to visit

$P_i$ : non-negative density of trash expected at each hotspot $i$ (higher density = more desirable) These values are normalized to be from [0, 100]

$\lambda_i$ : sensory learning rate, given by $\frac{obs_r^2 \pi}{A_i}$, where $obs_r^2 \pi$ = UUV sensing rate, 
and $A_i$ = area of hotspot

$t_i$ : total amount of time spent at hotspot $i$

$h_ij = {0, 0.5, 1, 1.5, ... , T_{max}}$ : timeslot that the path from hotspot $i$ to $j$ is taken at. Timeslots 
are every 30 minutes from 0 to $T_{max}$

$[O_i, C_i]$ : Start and end times to spend at the node

$d_{i,j,t}$ : non-negative travel time/cost between node $i$ to $j$ during timeslot $t$

$[0, T_{max}]$ : start and end times are assigned this time window to limit time to $T_{max}$. Service times are 
included in the travel times

$x_{ij} \in \{0,1\}$ : 1 if uuv travels from node $i$ to $j$

$x_n$ : 1 if hotspot is visited by the uuv

$s_i$ : start time of the service at node $i$

$L$ : Large constant



In [1]:
import sys
from collections import defaultdict
import xlrd
import gurobipy as gp
from gurobipy import GRB

import numpy as np
import math

np.random.seed(248745)

In [2]:
class Hotspot():
    def __init__(self, number, profits, area, uuv_obs_rate):
        self.number = number
        self.profits = profits
        self.area = area
        self.lam = uuv_obs_rate/area

In [3]:
def reward_func(search_time_min, h_lam):
    '''
    search_time_min = time in minutes searching the hotspot
    h_lam = lamda value of the hotspot
    '''
    return 1-math.exp(-h_lam*search_time_min)

In [4]:
##import all hotspot to hotspot travel time over all time intervals
with open('./../santa_monica_energy_costs.npy', 'rb') as f:
    transit_times = np.load(f)

##build all the hotspot data
uuv_obs_rate = (5**2)*math.pi #arbitrarily selected observation rate
all_hotspots = [Hotspot(0, np.random.randint(0,100), np.random.randint(500,1000), uuv_obs_rate),
                Hotspot(1, np.random.randint(0,100), np.random.randint(500,1000), uuv_obs_rate),
                Hotspot(2, np.random.randint(0,100), np.random.randint(500,1000), uuv_obs_rate),
                Hotspot(3, np.random.randint(0,100), np.random.randint(500,1000), uuv_obs_rate),
                Hotspot(4, np.random.randint(0,100), np.random.randint(500,1000), uuv_obs_rate),
                Hotspot(5, np.random.randint(0,100), np.random.randint(500,1000), uuv_obs_rate)]

In [39]:
##Build useful data structures
all_time_hrs = 3*24
timeslots = np.arange(0, all_time_hrs, 0.5).tolist()

hotspot_names = np.arange(len(all_hotspots)).tolist()
hspt_profits = {hsp.number: hsp.profits for hsp in all_hotspots}
hspt_lam     = {hsp.number: hsp.lam for hsp in all_hotspots}
hspt_area    = {hsp.number: hsp.area for hsp in all_hotspots}


Tmax = 20000000 ##This is the maximium amount of energy allowed to expend in the whole mission
# ept = 75*60.0 ##energy(k)/time (min)
ept = 75


print ("Profits")
print (hspt_profits)
print ("Lamda")
print (hspt_lam)
print ("Area")
print (hspt_area)

time_horizon = 8/0.5  ##8hrs/0.5
horizon_start = 0
horizon_end = int(time_horizon)

##start the for loop

##Take the average of all the values over the time horizon
transit_times_th = transit_times[:, :, horizon_start:horizon_end]
transit_mean_costs = np.empty((len(hotspot_names), len(hotspot_names)))
for i in hotspot_names:
    transit_mean_costs[i, :] = np.mean(transit_times_th[i,:], axis=1)

print ("transit_costs: ", transit_mean_costs)
##TOOD: multiple/add by some value to add noise to values further into the future
## transit_costs *= some function


##Create the model
model = gp.Model("oceans")

##Decision variables
#x_ij = binary variable if route is taken fron hotspot i to j
path_taken = model.addVars(hotspot_names, hotspot_names, vtype=GRB.BINARY, name="path_taken")

#x_n = binary variable if hotspot i has been visited
visited = model.addVars(hotspot_names, vtype=GRB.BINARY, name="visited")

##how much time to spend at each hotspot
ti = model.addVars(hotspot_names, vtype=GRB.CONTINUOUS, name="survey_time")

## Schedule job start/end times
# si = model.addVars(hotspot_names, vtype=GRB.CONTINUOUS, lb=0.0, name="start_time")
# ei = model.addVars(hotspot_names, vtype=GRB.CONTINUOUS, lb=0.0, name='end_time')

# adjust_si = model.addVars(hotspot_names, vtype=GRB.CONTINUOUS, name="adjusted_start")

##Constraints
## A location may only be selected once for all time slots
# for i in hotspot_names:
#     model.addConstr(visited[i] <= 1, name="loc_check")
    
for i in hotspot_names:
        model.addConstr((sum(path_taken[i,j] for j in hotspot_names) <= 1), name="paths")

## A path may only be selected once for all time slots
# for i in hotspot_names:
#     for j in hotspot_names:
#         model.addConstr( (sum(path_taken[i,j]) <= 1) , name="path_check")


## If the path is selected, then the hotspot has been visited
# for i in range(len(hotspot_names)):
#     for j in range(len(hotspot_names)):
#         if sum(path_taken[i,j,h] for h in timeslots) == 1:
#             visited[j, h] = 1

# for i in hotspot_names:
#     for j in hotspot_names:
#         model.addConstr( (
#                           visited[i] == path_taken[i,j]
#                          ), name="link_path_loc")


## Ensure 𝑇𝑚𝑎𝑥 is not exceeded
# ∑𝑛∈𝑁 𝑡𝑖⋅𝑥𝑛 + ∑𝑖∈𝑁 ∑𝑗∈𝑁 ∑ℎ∈𝑇_𝑚𝑎𝑥  𝑑𝑖𝑗ℎ⋅𝑥𝑖𝑗ℎ ≤ 𝑇𝑚𝑎𝑥
# convert dijh to energy/time

# for i in hotspot_names:
#     for j in hotspot_names:
#         model.addConstr( (
#                           (ti[i] * ept * visited[i]) + (transit_mean_costs[i,j] * path_taken[i,j]) <= Tmax 
#                          ) , name="tmax_check")
        
for i in hotspot_names:
        model.addConstr( (
                          (ti[i] * ept * visited[i]) 
                        + sum(transit_mean_costs[i,j] * path_taken[i,j] for j in hotspot_names) <= Tmax 
                         ) , name="tmax_check")
        
#         model.addConstr( sum( ti[i] * ept * visited[i,h] for h in timeslots) <= Tmax, name="test")
#         model.addConstr( sum( transit_times[i, j, h] * path_taken[i,j,timeslots[h]] for h in range(len(timeslots)) ) <= Tmax, name="test2")

        
## Restrict start service time to time window
## Start time of hotspot j may not be before (start time hotspot i + traveling time from i to j + time spent at hotspot i)
##End time must also end before the beginning of the next start time (is this necessary?)
# 𝑠𝑖+𝑡𝑖+𝑑𝑖𝑗ℎ≤𝑠𝑗
# for hotspots serviced:
#     make sure next hotspot doesn't overlap with the previous hotspot
#     How to assign the hotspot to be next?

# M = 3000
# for i in hotspot_names:
#     for j in hotspot_names:
# #         model.addConstr( (si[i] + ti[i] + sum(transit_times[i,j,h] for h in range(len(timeslots)))
# #                         - M*sum(1 - sum(path_taken[i,j,h] for h in timeslots)) <= si[j] ), name='time_windows' )
#         model.addConstr(( si[i] + ti[i] + M*(transit_mean_costs[i,j] * path_taken[i,j]) <= si[j] 
#                         ), name='time_windows' )

##If a path is chosen at that time, then update its starting timeslots
# for i in hotspot_names:
#     model.addConstr((si[i] >= adjust_si[i]), name="start_adjust")



Profits
{0: 46, 1: 37, 2: 70, 3: 8, 4: 47, 5: 43}
Lamda
{0: 0.0991664347724051, 1: 0.1005631451213122, 2: 0.09904138252174632, 3: 0.1473542520445494, 4: 0.07981688652413092, 5: 0.11432287676818753}
Area
{0: 792, 1: 781, 2: 793, 3: 533, 4: 984, 5: 687}
transit_costs:  [[9.99999990e+07 0.00000000e+00 0.00000000e+00 1.49869778e+03
  0.00000000e+00 0.00000000e+00]
 [9.35510198e+03 9.99999990e+07 7.09797576e+03 1.10286930e+04
  7.67790731e+03 4.76841889e+03]
 [2.16501521e+03 0.00000000e+00 9.99999990e+07 3.76238845e+03
  6.10327362e+02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 9.99999990e+07
  0.00000000e+00 0.00000000e+00]
 [1.64693185e+03 0.00000000e+00 0.00000000e+00 3.20906965e+03
  9.99999990e+07 0.00000000e+00]
 [4.42275282e+03 0.00000000e+00 2.22104405e+03 6.06576817e+03
  2.85903462e+03 9.99999990e+07]]


In [72]:
##Objective function
for i in hotspot_names:
#     for j in hotspot_names:
#             model.setObjective(hspt_profits[i] * path_taken[i,j] * (1 - math.exp(-hspt_lam[i] * ti[i])), GRB.MAXIMIZE)
        model.setObjective( hspt_profits[i]*hspt_lam[i] * ti[i] * sum(path_taken[i,j] for j in hotspot_names), GRB.MAXIMIZE)
    
# model.setObjective(sum(ti[i] for i in hotspot_names), GRB.MAXIMIZE)
                               
# model.setObjective(sum(hspt_profits[i] * hspt_lam[i] * ti[i] for i in hotspot_names) , GRB.MAXIMIZE)

model.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (linux64)
Optimize a model with 0 rows, 48 columns and 0 nonzeros
Model fingerprint: 0xa50dddbe
Model has 6 quadratic objective terms
Model has 6 quadratic constraints
Variable types: 6 continuous, 42 integer (42 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [8e+01, 8e+01]
  QLMatrix range   [6e+02, 1e+08]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e+01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [2e+07, 2e+07]

Loaded MIP start from previous solve with objective -0

Presolve removed 0 rows and 42 columns
Presolve time: 0.00s
Presolved: 11 rows, 21 columns, 30 nonzeros
Presolved model has 10 SOS constraint(s)
Variable types: 11 continuous, 10 integer (10 binary)

Root relaxation: unbounded, 0 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

In [68]:
# for i in hotspot_names:
#     print(sum(path_taken[i,j] for j in hotspot_names))
    
(hspt_profits[5] * hspt_lam[5] * ti[5]) * (path_taken[5,j] for j in hotspot_names)

GurobiError: Invalid argument to LinExpr multiplication

In [54]:
'''
#x_ij = binary variable if route is taken fron hotspot i to j
path_taken = model.addVars(hotspot_names, hotspot_names, vtype=GRB.BINARY, name="path_taken")

#x_n = binary variable if hotspot i has been visited
visited = model.addVars(hotspot_names, vtype=GRB.BINARY, name="visited")

##how much time to spend at each hotspot
ti = model.addVars(hotspot_names, vtype=GRB.CONTINUOUS, name="survey_time")
'''

for i in hotspot_names:
    for j in hotspot_names:
        print (path_taken[i,j])

print ("\nvisited: ")
for i in hotspot_names:
    print(visited[i])
    
print ("\ntime at hotspots")
for i in hotspot_names:
    print ("ti: ", ti[i])

<gurobi.Var path_taken[0,0] (value 0.0)>
<gurobi.Var path_taken[0,1] (value 0.0)>
<gurobi.Var path_taken[0,2] (value 0.0)>
<gurobi.Var path_taken[0,3] (value 0.0)>
<gurobi.Var path_taken[0,4] (value 0.0)>
<gurobi.Var path_taken[0,5] (value 0.0)>
<gurobi.Var path_taken[1,0] (value 0.0)>
<gurobi.Var path_taken[1,1] (value 0.0)>
<gurobi.Var path_taken[1,2] (value 0.0)>
<gurobi.Var path_taken[1,3] (value 0.0)>
<gurobi.Var path_taken[1,4] (value 0.0)>
<gurobi.Var path_taken[1,5] (value 0.0)>
<gurobi.Var path_taken[2,0] (value 0.0)>
<gurobi.Var path_taken[2,1] (value 0.0)>
<gurobi.Var path_taken[2,2] (value 0.0)>
<gurobi.Var path_taken[2,3] (value 0.0)>
<gurobi.Var path_taken[2,4] (value 0.0)>
<gurobi.Var path_taken[2,5] (value 0.0)>
<gurobi.Var path_taken[3,0] (value 0.0)>
<gurobi.Var path_taken[3,1] (value 0.0)>
<gurobi.Var path_taken[3,2] (value 0.0)>
<gurobi.Var path_taken[3,3] (value 0.0)>
<gurobi.Var path_taken[3,4] (value 0.0)>
<gurobi.Var path_taken[3,5] (value 0.0)>
<gurobi.Var path

In [ ]:
##Build useful data structures
all_time_hrs = 3*24
timeslots = np.arange(0, all_time_hrs, 0.5).tolist()

hotspot_names = np.arange(len(all_hotspots)).tolist()
hspt_profits = {hsp.number: hsp.profits for hsp in all_hotspots}
hspt_lam     = {hsp.number: hsp.lam for hsp in all_hotspots}
hspt_area    = {hsp.number: hsp.area for hsp in all_hotspots}


Tmax = 2 000 ##This is the maximium amount of energy allowed to expend in the whole mission
ept = 75*60.0 ##energy(k)/time (min)


print ("Profits")
print (hspt_profits)
print ("Lamda")
print (hspt_lam)
print ("Area")
print (hspt_area)

time_horizon = 8/0.5  ##8hrs/0.5
horizon_start = 0.0
horizon_end = time_horizon

##start the for loop

transit_times_th = transit_times[:, :, horizon_start:horizon_end]
travel_costs = np.empty((len(hotspot_names), len(hotspot_names)))
for i in hotspot_names:
    travel_costs[i, :] = np.mean(transit_times_th, axis=1)

print ("transit_costs: ", transit_costs)
##TOOD: multiple/add by some value to add noise to values further into the future



##Create the model
model = gp.Model("oceans")

##Decision variables
#x_ijt = binary variable if route is taken fron hotspot i to j at timeslot h
path_taken = model.addVars(hotspot_names, hotspot_names, travel_costs, vtype=GRB.BINARY, name="path_taken")

#x_n = binary variable if hotspot i has been visited
visited = model.addVars(hotspot_names, timeslots, vtype=GRB.BINARY, name="visited")

##how much time to spend at each hotspot
ti = model.addVars(hotspot_names, vtype=GRB.CONTINUOUS, name="survey_time")

## Schedule job start/end times
si = model.addVars(hotspot_names, vtype=GRB.CONTINUOUS, lb=0.0, name="start_time")
ei = model.addVars(hotspot_names, vtype=GRB.CONTINUOUS, lb=0.0, name='end_time')

adjust_si = model.addVars(hotspot_names, vtype=GRB.CONTINUOUS, name="adjusted_start")

##Constraints
## A location may only be selected once for all time slots
for i in hotspot_names:
    model.addConstr( (sum(visited[i,h] for h in timeslots)) <= 1, name="loc_check")

## A path may only be selected once for all time slots
for i in hotspot_names:
    for j in hotspot_names:
        model.addConstr( (sum(path_taken[i,j,h] for h in timeslots) <= 1) , name="path_check")


## If the path is selected, then the hotspot has been visited
# for i in range(len(hotspot_names)):
#     for j in range(len(hotspot_names)):
#         if sum(path_taken[i,j,h] for h in timeslots) == 1:
#             visited[j, h] = 1

for i in hotspot_names:
    for j in hotspot_names:
        model.addConstr( (
                          sum(visited[i,h] for h in timeslots) == sum(path_taken[i,j,h] for h in timeslots)
                         ), name="link_path_loc")


## Ensure 𝑇𝑚𝑎𝑥 is not exceeded
# ∑𝑛∈𝑁 𝑡𝑖⋅𝑥𝑛 + ∑𝑖∈𝑁 ∑𝑗∈𝑁 ∑ℎ∈𝑇_𝑚𝑎𝑥  𝑑𝑖𝑗ℎ⋅𝑥𝑖𝑗ℎ ≤ 𝑇𝑚𝑎𝑥
# convert dijh to energy/time

for i in hotspot_names:
    for j in hotspot_names:
        model.addConstr( (
                          (sum( ti[i] * ept * visited[i,h] for h in timeslots)
                         + sum( transit_times[i,j,h] * path_taken[i,j,timeslots[h]]
                               for h in range(len(timeslots))) <= Tmax ) 
                         ) , name="tmax_check")
        
#         model.addConstr( sum( ti[i] * ept * visited[i,h] for h in timeslots) <= Tmax, name="test")
#         model.addConstr( sum( transit_times[i, j, h] * path_taken[i,j,timeslots[h]] for h in range(len(timeslots)) ) <= Tmax, name="test2")

        
## Restrict start service time to time window
## Start time of hotspot j may not be before (start time hotspot i + traveling time from i to j + time spent at hotspot i)
##End time must also end before the beginning of the next start time (is this necessary?)
# 𝑠𝑖+𝑡𝑖+𝑑𝑖𝑗ℎ≤𝑠𝑗
# for hotspots serviced:
#     make sure next hotspot doesn't overlap with the previous hotspot
#     How to assign the hotspot to be next?

M = 3000
for i in hotspot_names:
    for j in hotspot_names:
#         model.addConstr( (si[i] + ti[i] + sum(transit_times[i,j,h] for h in range(len(timeslots)))
#                         - M*sum(1 - sum(path_taken[i,j,h] for h in timeslots)) <= si[j] ), name='time_windows' )
        model.addConstr(( si[i] + ti[i] + sum(transit_times[i,j,h] for h in range(len(timeslots))) +
                          sum( transit_times[i,j,h] * path_taken[i,j,timeslots[h]]
                               for h in range(len(timeslots))) <= si[j] 
                        ), name='time_windows' )

##If a path is chosen at that time, then update its starting timeslots
for i in hotspot_names:
    model.addConstr((si[i] >= adjust_si[i]), name="start_adjust")

In [75]:


# Create dictionaries to capture factory supply limits, depot throughput limits, and customer demand.

supply = dict({'Liverpool': 150000,
               'Brighton': 200000})

through = dict({'Newcastle': 70000,
                'Birmingham': 50000,
                'London': 100000,
                'Exeter': 40000})

demand = dict({'C1': 50000,
               'C2': 10000,
               'C3': 40000,
               'C4': 35000,
               'C5': 60000,
               'C6': 20000})

# Create a dictionary to capture shipping costs.

arcs, cost = gp.multidict({
    ('Liverpool', 'Newcastle'): 0.5,
    ('Liverpool', 'Birmingham'): 0.5,
    ('Liverpool', 'London'): 1.0,
    ('Liverpool', 'Exeter'): 0.2,
    ('Liverpool', 'C1'): 1.0,
    ('Liverpool', 'C3'): 1.5,
    ('Liverpool', 'C4'): 2.0,
    ('Liverpool', 'C6'): 1.0,
    ('Brighton', 'Birmingham'): 0.3,
    ('Brighton', 'London'): 0.5,
    ('Brighton', 'Exeter'): 0.2,
    ('Brighton', 'C1'): 2.0,
    ('Newcastle', 'C2'): 1.5,
    ('Newcastle', 'C3'): 0.5,
    ('Newcastle', 'C5'): 1.5,
    ('Newcastle', 'C6'): 1.0,
    ('Birmingham', 'C1'): 1.0,
    ('Birmingham', 'C2'): 0.5,
    ('Birmingham', 'C3'): 0.5,
    ('Birmingham', 'C4'): 1.0,
    ('Birmingham', 'C5'): 0.5,
    ('London', 'C2'): 1.5,
    ('London', 'C3'): 2.0,
    ('London', 'C5'): 0.5,
    ('London', 'C6'): 1.5,
    ('Exeter', 'C3'): 0.2,
    ('Exeter', 'C4'): 1.5,
    ('Exeter', 'C5'): 0.5,
    ('Exeter', 'C6'): 1.5
})



model = gp.Model('SupplyNetworkDesign')
flow = model.addVars(arcs, obj=cost, name="flow")

print (cost)

{('Liverpool', 'Newcastle'): 0.5, ('Liverpool', 'Birmingham'): 0.5, ('Liverpool', 'London'): 1.0, ('Liverpool', 'Exeter'): 0.2, ('Liverpool', 'C1'): 1.0, ('Liverpool', 'C3'): 1.5, ('Liverpool', 'C4'): 2.0, ('Liverpool', 'C6'): 1.0, ('Brighton', 'Birmingham'): 0.3, ('Brighton', 'London'): 0.5, ('Brighton', 'Exeter'): 0.2, ('Brighton', 'C1'): 2.0, ('Newcastle', 'C2'): 1.5, ('Newcastle', 'C3'): 0.5, ('Newcastle', 'C5'): 1.5, ('Newcastle', 'C6'): 1.0, ('Birmingham', 'C1'): 1.0, ('Birmingham', 'C2'): 0.5, ('Birmingham', 'C3'): 0.5, ('Birmingham', 'C4'): 1.0, ('Birmingham', 'C5'): 0.5, ('London', 'C2'): 1.5, ('London', 'C3'): 2.0, ('London', 'C5'): 0.5, ('London', 'C6'): 1.5, ('Exeter', 'C3'): 0.2, ('Exeter', 'C4'): 1.5, ('Exeter', 'C5'): 0.5, ('Exeter', 'C6'): 1.5}
